In [1]:
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv


Using CPU. Note: This module is much faster with a GPU.


In [17]:
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv


results = {}

mot_tracker = Sort()

# load models
coco_model = YOLO('yolo11n.pt')
license_plate_detector = YOLO('detection_model.pt')

# load video
cap = cv2.VideoCapture('./test2.mp4')

vehicles = [2, 3, 5, 7]

# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}

# write results
write_csv(results, './test.csv')


0: 480x640 2 cars, 74.5ms
Speed: 2.4ms preprocess, 74.5ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 63.8ms
Speed: 1.6ms preprocess, 63.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 67.9ms
Speed: 1.7ms preprocess, 67.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 57.2ms
Speed: 2.2ms preprocess, 57.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 1 train, 56.7ms
Speed: 2.9ms preprocess, 56.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 50.2ms
Speed: 2.3ms preprocess, 50.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 1 train, 63.1ms
Speed: 2.1ms preprocess, 63.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 67.1ms
Speed: 2.0ms preprocess, 67.1ms inference, 0.4ms po

In [18]:
import csv
import numpy as np
from scipy.interpolate import interp1d


def interpolate_bounding_boxes(data):
    # Extract necessary data columns from input data
    frame_numbers = np.array([int(row['frame_nmr']) for row in data])
    car_ids = np.array([int(float(row['car_id'])) for row in data])
    car_bboxes = np.array([list(map(float, row['car_bbox'][1:-1].split())) for row in data])
    license_plate_bboxes = np.array([list(map(float, row['license_plate_bbox'][1:-1].split())) for row in data])

    interpolated_data = []
    unique_car_ids = np.unique(car_ids)
    for car_id in unique_car_ids:

        frame_numbers_ = [p['frame_nmr'] for p in data if int(float(p['car_id'])) == int(float(car_id))]
        print(frame_numbers_, car_id)

        # Filter data for a specific car ID
        car_mask = car_ids == car_id
        car_frame_numbers = frame_numbers[car_mask]
        car_bboxes_interpolated = []
        license_plate_bboxes_interpolated = []

        first_frame_number = car_frame_numbers[0]
        last_frame_number = car_frame_numbers[-1]

        for i in range(len(car_bboxes[car_mask])):
            frame_number = car_frame_numbers[i]
            car_bbox = car_bboxes[car_mask][i]
            license_plate_bbox = license_plate_bboxes[car_mask][i]

            if i > 0:
                prev_frame_number = car_frame_numbers[i-1]
                prev_car_bbox = car_bboxes_interpolated[-1]
                prev_license_plate_bbox = license_plate_bboxes_interpolated[-1]

                if frame_number - prev_frame_number > 1:
                    # Interpolate missing frames' bounding boxes
                    frames_gap = frame_number - prev_frame_number
                    x = np.array([prev_frame_number, frame_number])
                    x_new = np.linspace(prev_frame_number, frame_number, num=frames_gap, endpoint=False)
                    interp_func = interp1d(x, np.vstack((prev_car_bbox, car_bbox)), axis=0, kind='linear')
                    interpolated_car_bboxes = interp_func(x_new)
                    interp_func = interp1d(x, np.vstack((prev_license_plate_bbox, license_plate_bbox)), axis=0, kind='linear')
                    interpolated_license_plate_bboxes = interp_func(x_new)

                    car_bboxes_interpolated.extend(interpolated_car_bboxes[1:])
                    license_plate_bboxes_interpolated.extend(interpolated_license_plate_bboxes[1:])

            car_bboxes_interpolated.append(car_bbox)
            license_plate_bboxes_interpolated.append(license_plate_bbox)

        for i in range(len(car_bboxes_interpolated)):
            frame_number = first_frame_number + i
            row = {}
            row['frame_nmr'] = str(frame_number)
            row['car_id'] = str(car_id)
            row['car_bbox'] = ' '.join(map(str, car_bboxes_interpolated[i]))
            row['license_plate_bbox'] = ' '.join(map(str, license_plate_bboxes_interpolated[i]))

            if str(frame_number) not in frame_numbers_:
                # Imputed row, set the following fields to '0'
                row['license_plate_bbox_score'] = '0'
                row['license_number'] = '0'
                row['license_number_score'] = '0'
            else:
                # Original row, retrieve values from the input data if available
                original_row = [p for p in data if int(p['frame_nmr']) == frame_number and int(float(p['car_id'])) == int(float(car_id))][0]
                row['license_plate_bbox_score'] = original_row['license_plate_bbox_score'] if 'license_plate_bbox_score' in original_row else '0'
                row['license_number'] = original_row['license_number'] if 'license_number' in original_row else '0'
                row['license_number_score'] = original_row['license_number_score'] if 'license_number_score' in original_row else '0'

            interpolated_data.append(row)

    return interpolated_data


# Load the CSV file
with open('test.csv', 'r') as file:
    reader = csv.DictReader(file)
    data = list(reader)

# Interpolate missing data
interpolated_data = interpolate_bounding_boxes(data)

# Write updated data to a new CSV file
header = ['frame_nmr', 'car_id', 'car_bbox', 'license_plate_bbox', 'license_plate_bbox_score', 'license_number', 'license_number_score']
with open('test_interpolated.csv', 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=header)
    writer.writeheader()
    writer.writerows(interpolated_data)


['238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '249', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '278', '279', '281', '282', '283', '285', '293', '294', '295', '297', '298', '299', '300', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '314', '315', '316', '317', '318', '319', '320', '321', '322', '325', '330', '332', '335', '336', '337', '339', '340', '341', '350', '351', '353', '358', '368', '370', '371', '372', '373', '378', '385', '386', '395', '407', '408', '409', '415', '421', '428', '438', '439', '459', '463', '464', '465', '466', '467', '468', '562', '567', '568', '578', '579', '592', '601', '619', '620'] 84


In [19]:
import ast

import cv2
import numpy as np
import pandas as pd


def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=10, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv2.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv2.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv2.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv2.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv2.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv2.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv2.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img


results = pd.read_csv('./test_interpolated.csv')

# load video
video_path = 'test2.mp4'
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./out.mp4', fourcc, fps, (width, height))

license_plate = {}
for car_id in np.unique(results['car_id']):
    max_ = np.amax(results[results['car_id'] == car_id]['license_number_score'])
    license_plate[car_id] = {'license_crop': None,
                             'license_plate_number': results[(results['car_id'] == car_id) &
                                                             (results['license_number_score'] == max_)]['license_number'].iloc[0]}
    cap.set(cv2.CAP_PROP_POS_FRAMES, results[(results['car_id'] == car_id) &
                                             (results['license_number_score'] == max_)]['frame_nmr'].iloc[0])
    ret, frame = cap.read()

    x1, y1, x2, y2 = ast.literal_eval(results[(results['car_id'] == car_id) &
                                              (results['license_number_score'] == max_)]['license_plate_bbox'].iloc[0].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

    license_crop = frame[int(y1):int(y2), int(x1):int(x2), :]
    license_crop = cv2.resize(license_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

    license_plate[car_id]['license_crop'] = license_crop


frame_nmr = -1

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
while ret:
    ret, frame = cap.read()
    frame_nmr += 1
    if ret:
        df_ = results[results['frame_nmr'] == frame_nmr]
        for row_indx in range(len(df_)):
            # draw car
            car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                        line_length_x=200, line_length_y=200)

            # draw license plate
            x1, y1, x2, y2 = ast.literal_eval(df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

            # crop license plate
            license_crop = license_plate[df_.iloc[row_indx]['car_id']]['license_crop']

            H, W, _ = license_crop.shape

            try:
                frame[int(car_y1) - H - 100:int(car_y1) - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = license_crop

                frame[int(car_y1) - H - 400:int(car_y1) - H - 100,
                      int((car_x2 + car_x1 - W) / 2):int((car_x2 + car_x1 + W) / 2), :] = (255, 255, 255)

                (text_width, text_height), _ = cv2.getTextSize(
                    license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                    cv2.FONT_HERSHEY_SIMPLEX,
                    4.3,
                    17)

                cv2.putText(frame,
                            license_plate[df_.iloc[row_indx]['car_id']]['license_plate_number'],
                            (int((car_x2 + car_x1 - text_width) / 2), int(car_y1 - H - 250 + (text_height / 2))),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            4.3,
                            (0, 0, 0),
                            17)

            except:
                pass

        out.write(frame)
        frame = cv2.resize(frame, (1280, 720))

        # cv2.imshow('frame', frame)
        # cv2.waitKey(0)

out.release()
cap.release()


In [ ]:
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv


results = {}

mot_tracker = Sort()

# load models
coco_model = YOLO('yolo11n.pt')
license_plate_detector = YOLO('detection_model.pt')

# load video
cap = cv2.VideoCapture('./test2.mp4')

vehicles = [2, 3, 5, 7]

# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}

# write results
write_csv(results, './test.csv')


0: 480x640 2 cars, 74.5ms
Speed: 2.4ms preprocess, 74.5ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 63.8ms
Speed: 1.6ms preprocess, 63.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 67.9ms
Speed: 1.7ms preprocess, 67.9ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 57.2ms
Speed: 2.2ms preprocess, 57.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 1 train, 56.7ms
Speed: 2.9ms preprocess, 56.7ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 50.2ms
Speed: 2.3ms preprocess, 50.2ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 cars, 1 train, 63.1ms
Speed: 2.1ms preprocess, 63.1ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 67.1ms
Speed: 2.0ms preprocess, 67.1ms inference, 0.4ms po

In [20]:
import ast
import cv2
import numpy as np
import pandas as pd


def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=10, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv2.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv2.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv2.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv2.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv2.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv2.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv2.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img


results = pd.read_csv('./test_interpolated.csv')

# load video
video_path = 'test2.mp4'
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./out.mp4', fourcc, fps, (width, height))

license_plate = {}
for car_id in np.unique(results['car_id']):
    try:
        # Check if license_number_score column exists and has valid data
        if 'license_number_score' not in results.columns or results[results['car_id'] == car_id]['license_number_score'].empty:
            print(f"Warning: No license_number_score for car_id {car_id}")
            continue
            
        max_ = np.amax(results[results['car_id'] == car_id]['license_number_score'])
        
        # Verify we have license number data
        if 'license_number' not in results.columns or results[(results['car_id'] == car_id) & 
                                               (results['license_number_score'] == max_)]['license_number'].empty:
            print(f"Warning: No license_number for car_id {car_id}")
            continue
            
        license_number = results[(results['car_id'] == car_id) & 
                                (results['license_number_score'] == max_)]['license_number'].iloc[0]
        
        license_plate[car_id] = {
            'license_crop': None,
            'license_plate_number': license_number
        }
        
        frame_nmr = results[(results['car_id'] == car_id) & 
                          (results['license_number_score'] == max_)]['frame_nmr'].iloc[0]
        
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_nmr)
        ret, frame = cap.read()
        
        if not ret:
            print(f"Warning: Could not read frame {frame_nmr} for car_id {car_id}")
            continue

        bbox_str = results[(results['car_id'] == car_id) & 
                          (results['license_number_score'] == max_)]['license_plate_bbox'].iloc[0]
        x1, y1, x2, y2 = ast.literal_eval(bbox_str.replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ','))

        # Make sure coordinates are within frame bounds
        y1, y2 = max(0, int(y1)), min(frame.shape[0], int(y2))
        x1, x2 = max(0, int(x1)), min(frame.shape[1], int(x2))
        
        if x2 <= x1 or y2 <= y1:
            print(f"Warning: Invalid bbox dimensions for car_id {car_id}")
            continue
            
        license_crop = frame[y1:y2, x1:x2, :]
        license_crop = cv2.resize(license_crop, (int((x2 - x1) * 400 / (y2 - y1)), 400))

        license_plate[car_id]['license_crop'] = license_crop
        
    except Exception as e:
        print(f"Error processing car_id {car_id}: {e}")


frame_nmr = -1

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
while ret:
    ret, frame = cap.read()
    frame_nmr += 1
    if ret:
        df_ = results[results['frame_nmr'] == frame_nmr]
        for row_indx in range(len(df_)):
            try:
                # draw car
                car_bbox_str = df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ',')
                car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(car_bbox_str)
                draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                            line_length_x=200, line_length_y=200)

                # draw license plate
                license_bbox_str = df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ',')
                x1, y1, x2, y2 = ast.literal_eval(license_bbox_str)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

                # Get car_id and check if it exists in our dictionary
                car_id = df_.iloc[row_indx]['car_id']
                if car_id not in license_plate:
                    print(f"Warning: Car ID {car_id} not in license_plate dictionary (frame {frame_nmr})")
                    continue
                    
                # Get license crop and check if it exists
                license_crop = license_plate[car_id]['license_crop']
                if license_crop is None:
                    print(f"Warning: No license crop for car_id {car_id} (frame {frame_nmr})")
                    continue
                    
                # Get dimensions
                H, W, _ = license_crop.shape
                
                # Calculate positions
                top_y = int(car_y1) - H - 100
                bottom_y = int(car_y1) - 100
                left_x = int((car_x2 + car_x1 - W) / 2)
                right_x = int((car_x2 + car_x1 + W) / 2)
                
                # Ensure positions are within frame bounds
                if top_y < 0:
                    offset = abs(top_y)
                    top_y = 0
                    bottom_y = min(bottom_y + offset, frame.shape[0])
                    print(f"Adjusted top position for car_id {car_id} (frame {frame_nmr})")
                
                if left_x < 0:
                    left_x = 0
                    right_x = min(W, frame.shape[1])
                    print(f"Adjusted left position for car_id {car_id} (frame {frame_nmr})")
                    
                if right_x >= frame.shape[1]:
                    right_x = frame.shape[1] - 1
                    left_x = max(right_x - W, 0)
                    print(f"Adjusted right position for car_id {car_id} (frame {frame_nmr})")
                
                if bottom_y >= frame.shape[0]:
                    bottom_y = frame.shape[0] - 1
                    top_y = max(bottom_y - H, 0)
                    print(f"Adjusted bottom position for car_id {car_id} (frame {frame_nmr})")
                
                # Resize license_crop if needed to fit the available space
                actual_h = bottom_y - top_y
                actual_w = right_x - left_x
                
                if actual_h != H or actual_w != W:
                    if actual_h > 0 and actual_w > 0:
                        license_crop_resized = cv2.resize(license_crop, (actual_w, actual_h))
                        frame[top_y:bottom_y, left_x:right_x, :] = license_crop_resized
                        print(f"Resized license crop for car_id {car_id} (frame {frame_nmr})")
                else:
                    # Place license crop
                    frame[top_y:bottom_y, left_x:right_x, :] = license_crop
                
                # Calculate text background position
                text_top_y = int(car_y1) - H - 400
                text_bottom_y = int(car_y1) - H - 100
                
                # Adjust text background position if needed
                if text_top_y < 0:
                    offset = abs(text_top_y)
                    text_top_y = 0
                    text_bottom_y = min(text_bottom_y + offset, top_y)
                
                if text_bottom_y >= frame.shape[0]:
                    text_bottom_y = frame.shape[0] - 1
                    text_top_y = max(text_bottom_y - 300, 0)
                
                # Place white background for text
                bg_h = text_bottom_y - text_top_y
                bg_w = right_x - left_x
                
                if bg_h > 0 and bg_w > 0:
                    # Create white background
                    white_bg = np.ones((bg_h, bg_w, 3), dtype=np.uint8) * 255
                    frame[text_top_y:text_bottom_y, left_x:right_x, :] = white_bg
                    
                    # Add text
                    license_number = license_plate[car_id]['license_plate_number']
                    (text_width, text_height), _ = cv2.getTextSize(
                        license_number,
                        cv2.FONT_HERSHEY_SIMPLEX,
                        4.3,
                        17)
                    
                    # Calculate text position
                    text_x = int((car_x2 + car_x1 - text_width) / 2)
                    text_y = int((text_top_y + text_bottom_y + text_height) / 2)
                    
                    # Ensure text is within frame bounds
                    if text_x < 0:
                        text_x = left_x + 10
                    
                    # Draw text
                    cv2.putText(frame,
                                license_number,
                                (text_x, text_y),
                                cv2.FONT_HERSHEY_SIMPLEX,
                                4.3,
                                (0, 0, 0),
                                17)
                
            except Exception as e:
                print(f"Error processing row {row_indx} in frame {frame_nmr}: {e}")

        out.write(frame)
        frame = cv2.resize(frame, (1280, 720))
        
        # Uncomment to display frame
        # cv2.imshow('frame', frame)
        # cv2.waitKey(1)  # Changed from 0 to 1 for continuous playback

print("Video processing complete")
out.release()
cap.release()
cv2.destroyAllWindows()

Adjusted top position for car_id 84 (frame 238)
Adjusted left position for car_id 84 (frame 238)
Adjusted right position for car_id 84 (frame 238)
Resized license crop for car_id 84 (frame 238)
Adjusted top position for car_id 84 (frame 239)
Adjusted left position for car_id 84 (frame 239)
Adjusted right position for car_id 84 (frame 239)
Resized license crop for car_id 84 (frame 239)
Adjusted top position for car_id 84 (frame 240)
Adjusted left position for car_id 84 (frame 240)
Adjusted right position for car_id 84 (frame 240)
Resized license crop for car_id 84 (frame 240)
Adjusted top position for car_id 84 (frame 241)
Adjusted left position for car_id 84 (frame 241)
Adjusted right position for car_id 84 (frame 241)
Resized license crop for car_id 84 (frame 241)
Adjusted top position for car_id 84 (frame 242)
Adjusted left position for car_id 84 (frame 242)
Adjusted right position for car_id 84 (frame 242)
Resized license crop for car_id 84 (frame 242)
Adjusted top position for car_

In [21]:
import ast
import cv2
import numpy as np
import pandas as pd


def draw_border(img, top_left, bottom_right, color=(0, 255, 0), thickness=10, line_length_x=200, line_length_y=200):
    x1, y1 = top_left
    x2, y2 = bottom_right

    cv2.line(img, (x1, y1), (x1, y1 + line_length_y), color, thickness)  #-- top-left
    cv2.line(img, (x1, y1), (x1 + line_length_x, y1), color, thickness)

    cv2.line(img, (x1, y2), (x1, y2 - line_length_y), color, thickness)  #-- bottom-left
    cv2.line(img, (x1, y2), (x1 + line_length_x, y2), color, thickness)

    cv2.line(img, (x2, y1), (x2 - line_length_x, y1), color, thickness)  #-- top-right
    cv2.line(img, (x2, y1), (x2, y1 + line_length_y), color, thickness)

    cv2.line(img, (x2, y2), (x2, y2 - line_length_y), color, thickness)  #-- bottom-right
    cv2.line(img, (x2, y2), (x2 - line_length_x, y2), color, thickness)

    return img


results = pd.read_csv('./test_interpolated.csv')

# load video
video_path = 'test2.mp4'
cap = cv2.VideoCapture(video_path)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Specify the codec
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./out.mp4', fourcc, fps, (width, height))

license_plate = {}
for car_id in np.unique(results['car_id']):
    try:
        # Check if license_number_score column exists and has valid data
        if 'license_number_score' not in results.columns or results[results['car_id'] == car_id]['license_number_score'].empty:
            print(f"Warning: No license_number_score for car_id {car_id}")
            continue
            
        max_ = np.amax(results[results['car_id'] == car_id]['license_number_score'])
        
        # Verify we have license number data
        if 'license_number' not in results.columns or results[(results['car_id'] == car_id) & 
                                               (results['license_number_score'] == max_)]['license_number'].empty:
            print(f"Warning: No license_number for car_id {car_id}")
            continue
            
        license_number = results[(results['car_id'] == car_id) & 
                                (results['license_number_score'] == max_)]['license_number'].iloc[0]
        
        license_plate[car_id] = {
            'license_plate_number': license_number
        }
    except Exception as e:
        print(f"Error processing car_id {car_id}: {e}")


frame_nmr = -1

cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# read frames
ret = True
while ret:
    ret, frame = cap.read()
    frame_nmr += 1
    if ret:
        df_ = results[results['frame_nmr'] == frame_nmr]
        for row_indx in range(len(df_)):
            try:
                # draw car
                car_bbox_str = df_.iloc[row_indx]['car_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ',')
                car_x1, car_y1, car_x2, car_y2 = ast.literal_eval(car_bbox_str)
                draw_border(frame, (int(car_x1), int(car_y1)), (int(car_x2), int(car_y2)), (0, 255, 0), 25,
                            line_length_x=200, line_length_y=200)

                # draw license plate
                license_bbox_str = df_.iloc[row_indx]['license_plate_bbox'].replace('[ ', '[').replace('   ', ' ').replace('  ', ' ').replace(' ', ',')
                x1, y1, x2, y2 = ast.literal_eval(license_bbox_str)
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 12)

                # Get car_id and check if it exists in our dictionary
                car_id = df_.iloc[row_indx]['car_id']
                if car_id not in license_plate:
                    print(f"Warning: Car ID {car_id} not in license_plate dictionary (frame {frame_nmr})")
                    continue
                
                # Get license plate text
                license_number = license_plate[car_id]['license_plate_number']
                
                # Calculate text size
                (text_width, text_height), _ = cv2.getTextSize(
                    license_number,
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1.0,  # Smaller font size
                    3     # Thinner text
                )
                
                # Calculate position for the text (above the license plate)
                text_x = int((x1 + x2 - text_width) / 2)
                text_y = int(y1 - 20)  # 20 pixels above the license plate
                
                # Make sure text is within frame
                if text_y < text_height + 10:
                    text_y = y1 + int(y2 - y1) + text_height + 10  # Place below if not enough space above
                
                if text_x < 0:
                    text_x = x1
                
                # Create background for text (white rectangle)
                bg_padding = 8
                bg_x1 = text_x - bg_padding
                bg_y1 = text_y - text_height - bg_padding
                bg_x2 = text_x + text_width + bg_padding
                bg_y2 = text_y + bg_padding
                
                # Ensure background is within frame bounds
                bg_x1 = max(0, bg_x1)
                bg_y1 = max(0, bg_y1)
                bg_x2 = min(frame.shape[1] - 1, bg_x2)
                bg_y2 = min(frame.shape[0] - 1, bg_y2)
                
                # Draw white background
                cv2.rectangle(frame, 
                             (int(bg_x1), int(bg_y1)), 
                             (int(bg_x2), int(bg_y2)), 
                             (255, 255, 255), 
                             -1)  # Filled rectangle
                
                # Draw text
                cv2.putText(frame,
                            license_number,
                            (text_x, text_y),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1.0,  # Font scale
                            (0, 0, 0),  # Black color
                            3)  # Thickness
                
            except Exception as e:
                print(f"Error processing row {row_indx} in frame {frame_nmr}: {e}")

        out.write(frame)
        frame = cv2.resize(frame, (1280, 720))
        
        # Uncomment to display frame
        # cv2.imshow('frame', frame)
        # cv2.waitKey(1)  # Changed from 0 to 1 for continuous playback

print("Video processing complete")
out.release()
cap.release()
cv2.destroyAllWindows()

Video processing complete
